# Analyse de données Budget et Finances : données pour les paniers ERRD

## Paramétrage

In [ ]:
import os

import pandas as pd
CHEMIN_DOSSIER_DONNÉES = '../../../../données/DIAMANT'
DATE_FICHIER = '2022_09_01'
encoding = 'utf-8'


In [ ]:
ann_per_errd_eprd = pd.read_csv(
    os.path.join(CHEMIN_DOSSIER_DONNÉES, 'ANN_PER_ERRD_EPRD_' + DATE_FICHIER + '.CSV'),
    delimiter=";",
    encoding=encoding,
    dtype='str'
)

## Doublons

### On s'attend à avoir une seule valeur par (Finess, Année, Id Dépôt)

In [ ]:
ann_per_errd_eprd.head()

In [ ]:
ann_per_errd_eprd.duplicated(subset=["Finess", "Année", "Id Dépôt"]).sum()

On a bien unicité

### A-t-on un seul numéro de dépôt pour un FINESS et une année ?

In [ ]:
a_plus_d_une_donnée_par_finess_par_année = (ann_per_errd_eprd[['Finess', 'Année', "Id Dépôt"]].groupby(['Finess', 'Année']).nunique() > 1)

In [ ]:
a_plus_d_une_donnée_par_finess_par_année.sum() / ann_per_errd_eprd.groupby(['Finess', 'Année']).ngroups

Non : la plupart des établissements ont plusieurs numéros de dépôt pour une même année

In [ ]:
ann_per_errd_eprd[['Finess', 'Année', "Id Dépôt"]].groupby(['Finess', 'Année']).nunique().plot(
    kind= 'hist',
    title="Répartition du nombre de dépôts pour un même Finess et année",
    legend=False,
)

In [ ]:
ann_per_errd_eprd[['Finess', 'Année', "Id Dépôt"]].groupby(['Finess', 'Année']).nunique().value_counts().sort_index()

La plupart des Finess font l'objet de 2-3 dépots / an

In [ ]:
ann_per_errd_eprd.groupby('Année')['Finess'].nunique().plot(kind="bar", title="répartition des Finess par année")

De plus en plus d'établissements sont concernés par les paniers ERRD : on voit donc de plus en plus de numéros FINESS apparaître par année

In [ ]:
ann_per_errd_eprd.groupby('Année')['Id Dépôt'].nunique().plot(kind="bar", title="répartition des numéros de dépôt par année")

Le nombre de dépôts est plus stable, mais augmente en 2020

Le nombre de FINESS renseignés augmente plus que le nombre de dépôts.

## A-t-on des groupements stables ?

Un Id Dépôt correspond à une mise à jour d'un groupement d'ET. Est-ce que ces groupements restent stables, auquel cas on peut facilement dédoublonner nos données ?

In [ ]:
groups = ann_per_errd_eprd[['Finess', 'Année', "Id Dépôt"]].groupby(["Id Dépôt", "Année"])['Finess'].apply(set)

In [ ]:
unique_groups = groups.sort_index(ascending=False).drop_duplicates()

In [ ]:
intersecting_groups = []
for group1 in unique_groups.values:
    for group2 in unique_groups.values:
        if (group1 != group2) and group1.intersection(group2):
            intersecting_groups.append((group1, group2))

In [ ]:
len(intersecting_groups) / 2

On voit que, dans une même année, les groupes d'ET qui sont déposés ensemble évoluent. Lequel fait foi ?
Réponse de Nicolas : pour chaque année, on prend le numéro de dépôt le plus élevé renseigné pour un FINESS. Questions persistantes : j'ai l'impression que parfois les dépôts ont des données partielles, complétées par des dépôts suivants

## Le taux de vétusté et de CAF par FINESS

### Les données des dépôts par paniers ERRD

In [ ]:
ann_errd_ej = pd.read_csv(
    os.path.join(CHEMIN_DOSSIER_DONNÉES, 'ANN_ERRD_EJ_' + DATE_FICHIER + '.CSV'),
    delimiter=";",
    encoding=encoding,
    dtype={"Id Dépôt": str},
)


In [ ]:
ann_errd_ej.head()


Valeurs manquantes par colonne identifiante :

In [ ]:
ann_errd_ej[["Finess EJ", "Année", "Id Dépôt"]].isna().sum() / ann_errd_ej.shape[0]


On a près de 3% des données des dépôts qu'on ne peut pas lier à des établissements, parce que leur numéro de dépôt et leur numéro FINESS EJ sont manquants.

❓ **Question à DIAMANT** : pourquoi a-t-on autant de dépôts "orphelins" ?

Pour la suite, nous alons simplement ignorer les dépôts non-identifiables

### Données des paniers ERRD par FINESS

In [ ]:
ann_errd_ej = ann_errd_ej.dropna(subset=["Id Dépôt"]).astype(dtype={"Id Dépôt" : str})

In [ ]:
ann_errd_ej_par_finess = ann_per_errd_eprd.astype(dtype={"Id Dépôt" : str}).merge(ann_errd_ej, on="Id Dépôt", how="right")

ann_errd_ej_par_finess.head()

On vérifie la cohérence des données : a-t-on bien les mêmes années et FINESS EJ pour les mêmes Id Dépôt dans les deux fichiers ?

In [ ]:
(ann_errd_ej_par_finess["Année_x"].astype(int) != ann_errd_ej_par_finess["Année_y"].astype(int)).sum()

In [ ]:
(ann_errd_ej_par_finess["Finess EJ_x"].astype(str) != ann_errd_ej_par_finess["Finess EJ_y"].astype(str)).sum()


✅ Il n'y a pas d'incohérence sur les Années et FINESS EJ entre les deux fichiers

#### A-t-on beaucoup de dépôts pour un seul couple (Année, FINESS) ?

In [ ]:
ann_errd_ej_par_finess.groupby(["Finess", "Année_x"])['Id Dépôt'].nunique().hist()


In [ ]:
ann_errd_ej_par_finess.groupby(["Finess", "Année_x"])['Id Dépôt'].nunique().value_counts()


La grande majorité n'a qu'un seul dépôt par an, quelques rares cas en ont deux. Regardons de plus près à quoi ressemblent les données des établissements ayant deux dépôts pour une même année

#### Comment traiter les valeurs manquantes ?

Proposition :
- Prendre les dépôts passés pour remplir les valeurs manquantes dans le dernier dépôt
- Ne garder que le dernier dépôt

In [ ]:
dépôts_pour_un_même_établissement_et_année = ann_errd_ej_par_finess[ann_errd_ej_par_finess.duplicated(subset=["Finess", "Année_x"], keep=False)].sort_values(["Finess", "Année_x"])


In [ ]:
dépôts_pour_un_même_établissement_et_année_avec_données_manquantes_remplies = dépôts_pour_un_même_établissement_et_année.groupby(["Finess", "Année_x"]).ffill()

In [ ]:
dépôts_pour_un_même_établissement_et_année_avec_données_manquantes_remplies[["Finess", "Année_x"]] = dépôts_pour_un_même_établissement_et_année[["Finess", "Année_x"]]

In [ ]:
dépôts_pour_un_même_établissement_et_année_avec_données_manquantes_remplies.drop_duplicates(subset=["Finess", "Année_x"], keep='last')